# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,carlsbad,33.1581,-117.3506,25.98,82,100,2.06,US,1718215019
1,1,mananjary,-21.2167,48.3333,19.94,86,44,3.98,MG,1718215123
2,2,adamstown,-25.0660,-130.1015,21.08,77,78,4.88,PN,1718215124
3,3,magadan,59.5667,150.8000,8.63,85,82,1.95,RU,1718215125
4,4,happy valley-goose bay,53.3168,-60.3315,24.17,35,40,3.60,CA,1718215125


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.6
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 27)]  

# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
ideal_weather_df.head()

/var/folders/qf/1g7nyqn1365475byk1zh84fw0000gn/T/ipykernel_98775/664204864.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,carlsbad,33.1581,-117.3506,25.98,82,100,2.06,US,1718215019
2,2,adamstown,-25.0660,-130.1015,21.08,77,78,4.88,PN,1718215124
4,4,happy valley-goose bay,53.3168,-60.3315,24.17,35,40,3.60,CA,1718215125
6,6,uturoa,-16.7333,-151.4333,26.17,66,89,8.13,PF,1718215126
10,10,west island,-12.1568,96.8225,25.99,94,40,8.23,CC,1718215127


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ["Hotel Name"] = None

# Display sample data
hotel_df.head()

/var/folders/qf/1g7nyqn1365475byk1zh84fw0000gn/T/ipykernel_98775/3600660473.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df ["Hotel Name"] = None


,City,Country,Lat,Lng,Humidity,Hotel Name
0,carlsbad,US,33.1581,-117.3506,82,None
2,adamstown,PN,-25.0660,-130.1015,77,None
4,happy valley-goose bay,CA,53.3168,-60.3315,35,None
6,uturoa,PF,-16.7333,-151.4333,66,None
10,west island,CC,-12.1568,96.8225,94,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1

# Set up a parameters dictionary
params = {
    "categories": categories,
    "limit":limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df["Lat"][index]
    lng = hotel_df["Lng"][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carlsbad - nearest hotel: Carlsbad Inn Beach Resort
adamstown - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: Hotel North Two
uturoa - nearest hotel: Hawaiki Nui hotel
west island - nearest hotel: Cocos Village Bungalows
chengtangcun - nearest hotel: 滕州饭店
kismayo - nearest hotel: Kismayo Hotel
port mathurin - nearest hotel: Escale Vacances
arraial do cabo - nearest hotel: No hotel found
lorengau - nearest hotel: Seeadler Bay Hotel
bethel - nearest hotel: Hampton Inn Danbury
tabou - nearest hotel: hôtel le rochet
winnipeg - nearest hotel: Best Western Plus Downtown Winnipeg
hinthada - nearest hotel: Shwe Mi Hotel
udachny - nearest hotel: Вилюй
avarua - nearest hotel: Paradise Inn
fereydun kenar - nearest hotel: هتل کیش مهر
mogadishu - nearest hotel: Villa Rays Restaurant & Hotel
naze - nearest hotel: ビジネス旅館畠山
ilhabela - nearest hotel: Vila Kebaya
weno - nearest hotel: High Tide Hotel
puerto baquerizo moreno - nearest hotel: Dory's House
ribe

,City,Country,Lat,Lng,Humidity,Hotel Name
0,carlsbad,US,33.1581,-117.3506,82,Carlsbad Inn Beach Resort
2,adamstown,PN,-25.0660,-130.1015,77,No hotel found
4,happy valley-goose bay,CA,53.3168,-60.3315,35,Hotel North Two
6,uturoa,PF,-16.7333,-151.4333,66,Hawaiki Nui hotel
10,west island,CC,-12.1568,96.8225,94,Cocos Village Bungalows
...,...,...,...,...,...,...
549,voi,KE,-3.3945,38.5630,64,Fine Breeze Hotel
551,shakhtersk,UA,48.0625,38.5167,56,No hotel found
552,yeraliyev,KZ,43.2000,51.6500,64,Kuryk
555,nurlat,RU,54.4277,50.8051,71,Гостиница «Нурлат»


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",  
    color="City",  
    alpha=0.6,
    hover_cols=["Hotel Name", "Country"]  
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)